In [ ]:
import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from google.colab.patches import cv2_imshow # colab内で画像表示関数がうまく動かないので、パッチが提供されている

# Googleドライブへのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/ou_dip/"

In [ ]:
# 自分でカーネルを作る例

img = cv2.imread("sample.jpg", cv2.IMREAD_GRAYSCALE) # 画像読み込み（グレースケール）

kernel = np.ones((31,31),np.float32) / (31*31) # 31x31平均化フィルタ
dst = cv2.filter2D(img,-1,kernel) # ddepth=-1とした場合、入出力のchannel数はおなじになる

cv2_imshow(dst)

In [ ]:
# 自分でカーネルを作る例 その2

img = cv2.imread("sample.jpg", cv2.IMREAD_GRAYSCALE) # 画像読み込み（グレースケール）

kernel = np.array(([-1.0,0.0,1.0],
                   [-3.0,0.0,3.0],
                   [-1.0,0.0,1.0])) # Sobelフィルタ（横方向の勾配）

dst = cv2.filter2D(img,-1,kernel) # ddepth=-1とした場合、入出力のchannel数はおなじになる

cv2_imshow(dst)